In [1]:
import gc
import inspect

import MySQLdb as mysql


db_name = 'my_orm'
# тут мои данные, так что меняй

In [2]:
def init_db(host, user, password, db_name):
    db = mysql.connect(host=host, user=user,
                       password=password)
    cursor = db.cursor()
    
    cursor.execute('CREATE DATABASE IF NOT EXISTS `{0}`'.format(db_name))
    
    db.close()
    gc.collect()

In [3]:
# тут мои данные, так что меняй
init_db(
    'localhost',
    'root',
    'lumen23291633',
    db_name
)

In [4]:
class Validator:
    
    _min_val = 0
    _max_val = 0 
    
    def __init__(self, min_val=0, max_val=0):
        _min_val = min_val
        _max_val = max_val
        
    @classmethod
    def validateMax(cls, value):
        if isinstance(value, int):
            return value if value <= cls._max_val else cls._max_val
        return cls._max_val
    
    @classmethod
    def validateMin(cls, value):
        if isinstance(value, int):
            return value if value >= cls._min_val else cls._min_val
        return cls._min_val
        
    
    
class IntValidator(Validator):
    
    _min_val = -2**31
    _max_val = 2**31 - 1
    
    def __init__(self, min_val=-2**31, max_val=2**31 - 1):
        super().__init__(min_val, max_val)
    
    
class CharValidator(Validator):
    
    _min_val = 1
    _max_val = 8000 
    
    def __init__(self, min_val=1, max_val=8000):
        super().__init__(min_val, max_val)
    
        
class FloatValidator(Validator):
    
    _min_val = -1.79e+308  
    _max_val = 1.79e+308
    
    def __init__(self, min_val=-1.79e+308, max_val=1.79e+308):
        super()._init_(min_val, max_val)
        
        
class Field:
    
    def __init__(self, name='', cls=None, required=True, default=None, primary_key=False):
        self.name = '_' + name
        self.cls = cls
        self.required = required
        self.default = self.validate(default)
        self.primary_key = primary_key
        
    def __get__(self, instance, owner=None):
        if owner is None:
            return AttributeError('This field must be defined in a class!')
        return getattr(instance, self.name, self.default)
    
    def __set__(self, instance, value):
        instance.__dict__[self.name] = value
    
    def __delete__(self, instance):
        if instance is None:
            raise AttributeError()
        del instance.__dict__[self.name]
        
    def validate(self, value):
        if value is None and not self.required:
            return None
        return self.cls(value)
    
    def cls_validate(self, value):
        if not isinstance(value, self.cls):
            raise AttributeError('The value have to be {0}'.format(self.cls))
        return 
    
    @property
    def is_primary(self):
        return self.primary_key

    @property
    def is_required(self):
        return self.required
    
    @property
    def get_defaule(self):
        return self.default
        
class IntField(Field):
    
    def __init__(self, name='', min_value=None, max_value=None, **kwarg):
        self.min_value = IntValidator.validateMin(min_value)
        self.max_value = IntValidator.validateMax(max_value)
        super().__init__(name, int, **kwarg)
        
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
        
    def __set__(self, instance, value):
        super().cls_validate(value)
       
        if self.min_value > value or self.max_value < value:
            raise AttributeError(
                'Value have to lay between {0} and {1}'.format(
                    self.min_value,
                    self.max_value
                )
            )
        super().__set__(instance, value)

    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class CharField(Field):
    
    def __init__(self, name='', max_length=None, min_length=None, **kwarg):
        self.min_length = CharValidator.validateMin(min_length)
        self.max_length = CharValidator.validateMax(max_length)
        super().__init__(name, str, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        
        if self.min_length > len(value) or self.max_length < len(value):
            raise AttributeError(
                'Value have to lay between {0} and {1}'.format(
                    self.min_length,
                    self.max_lenght
                )
            )
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class BoolField(Field):
    
    def __init__(self, name='', **kwarg):
        super().__init__(name, bool, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class FloatField(Field):
    
    def __init__(self, name='', min_val=None, max_val=None, **kwarg):
        self.min_val = CharValidator.validateMin(min_val)
        self.max_val = CharValidator.validateMax(max_val)
        super().__init__(name, float, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
        
    def __delete__(self, instance):
        super().__delete__(instance)
        
        
class AutoField(IntField):
    
    def __init__(self, name='', **kwarg):
        kwarg['required'] = False
        if kwarg.get('default', None) is not None:
            raise AttributeError(
                'AutoField does not contain default value!'
            )
        super().__init__(name, min_value=0, max_value=2**31-1, **kwarg)
        
    def __get__(self, instance, owner=None):
        return super().__get__(instance, owner)
    
    def __set__(self, instance, value):
        super().cls_validate(value)
        super().__set__(instance, value)
    
    def __delete__(self, instance):
        super().__delete__(instance)

In [5]:
class MetaManager(type):
    
    def __new__(mcs, name, bases, namespace):
        return super().__new__(mcs, name, bases, namespace)
    
class Manager(metaclass=MetaManager):
    
    def __get__(self, instance, owner=None):
        return QuerySet(owner)
    
    def __delete__(self, instance):
        super().__delete__(instance)
        

In [6]:
class QuerySet:
    
    _query_set = list()
    
    def __init__(self, owner=None):
        if owner is None:
            raise AttributeError(
                'Owner is None'
            )
        self.cls = owner
    
    def create(self, **kwarg):
        return self.cls(**kwarg)
        
    def all(self):
        pass
    
    def get(self, **kwarg):
        pass
        

In [10]:
class MetaModel(type):
    
    def __new__(mcs, name, bases, namespace):
        super_new = super().__new__
        
        # To choose only bases class constructed by MetaModel
        parents = tuple([b for b in bases if isinstance(b, MetaModel)])
        if not parents:
            return super_new(mcs, name, bases, namespace)
        
        new_namespace = dict()
        module = namespace.pop('__module__', None)
        if module is not None:
            new_namespace['__module__'] = module
            
        classcell = namespace.pop('__classcell__', None)
        if classcell is not None:
            new_namespace['__classcell__'] = classcell

        new_class = super_new(mcs, name, parents, new_namespace)
        
        fields = {name: value for name, value in namespace.items() if isinstance(value, Field)}
        methods = {name: value for name, value in namespace.items() if inspect.isfunction(value)}
        
        fields_name = set(fields.keys())
        
        meta = namespace.pop('Meta', None)
        if meta is not None:
            if not hasattr(meta, 'table_name'):
                setattr(meta, 'table_name', name)
            # There shoulde change conditions
            if not hasattr(meta, 'fields'):
                setattr(meta, 'fields', fields_name)
        else:
            meta = type('Meta', tuple(), {'table_name' : name, 'fields' : fields_name})
        
        # To inherite the fields by MRO
        for base_cls in new_class.mro():
            for field_name, value in base_cls.__dict__.items():  
                if isinstance(value, Field):
                    fields.update({field_name: value})
                    continue
                if inspect.isfunction(value):
                    methods.update({field_name: value})
        
        
        fields = mcs._check_primary(fields)
        meta.fields.update(set(fields.keys()))
        
        fields.update(methods)
        fields.update(mcs._set_manager(namespace))

        
        # To add meta-data
        setattr(new_class, 'Meta', meta)
        
        # To set fields from base classes
        for attr_name, attr_value in fields.items():
            setattr(new_class, attr_name, attr_value)

        return new_class
    
    @classmethod
    def _check_primary(cls, namespace=None):
        if namespace is None:
            return None
        
        primary_dict = {name: value for name, value in namespace.items() if (value.is_primary and value.name != '_id')}
        if len(primary_dict) > 1:
            raise AttributeError(
                'Model has to have only one primary_key field'
            )
        elif len(primary_dict) == 1:
            namespace.pop('id')
            return namespace

        return namespace
    
    @classmethod
    def _set_manager(cls, namespace=None):
        if namespace is None:
            return None
        
        managers = {name: value for name, value in namespace.items() if isinstance(value, Manager)}
        if len(managers) == 0:
            managers.update({'objects': Manager()})
        if len(managers) > 1:
            raise AttributeError(
                'Table have to has only one Manager()'
            )
        return managers
            
        
class Model(metaclass=MetaModel):
    
    class Meta:
        table_name = 'Model'
        fields = {'id'}
    
    id = AutoField('id',primary_key=True)
    objects = Manager()
    
    def __init__(self, **kwargs):
        
        for attr_name, attr_value in kwargs.items():
            if attr_name in self.__class__.__dict__['Meta'].__dict__['fields']:
                setattr(self, attr_name, attr_value)
                
    def _get_primary_name_value(self):
        for name, value in self.__class__.__dict__.items():
            if isinstance(value, Field):
                if value.is_primary:
                    return name, getattr(self, name)
        raise RuntimeError('Model has no primary key!')
                
    def save(self):
        db = mysql.connect(host='localhost', user='root',
                           password='lumen23291633', db= 'my_orm')
        cursor = db.cursor()
        change_boolean = lambda x: int(x) if isinstance(x, bool) else x  
        add_ = lambda x : '"' + x + '"' if (isinstance(x, str)) else x
        types = {IntField: 'INT(10)', FloatField: 'FLOAT(53,8)',
                 CharField: 'CHAR(255)', BoolField: 'BOOLEAN', AutoField: 'INT(10)'}
        is_req = {False: '', True: 'NOT NULL', None: ''}
        is_prim = {False: '', True: 'PRIMARY KEY', None: ''}
        table_name = self.__class__.__dict__['Meta'].__dict__['table_name']
        
        #checking
        name, value = self._get_primary_name_value()
        print(name, value)
        if value is None:
            raise RuntimeError('Primary key must be not None')
        if (cursor.execute('SELECT * FROM {0} WHERE {1}={2}'.format(table_name,name,value))):
            update_query = 'UPDATE {0} SET {1} WHERE {2}'
            attr_list = [str(attr_name) + '=' + str(add_(change_boolean(getattr(self,attr_name)))) for attr_name, attr_value in self.__class__.__dict__.items() if isinstance(attr_value, Field) and not attr_value.is_primary]
            attr_list = ','.join(attr_list)
            update_query = update_query.format(
                table_name,
                attr_list,
                '{0}={1}'.format(name, value)
            )
            print(update_query)
            cursor.execute(update_query)
            db.commit()
            db.close()
            gc.collect()
            return
        
        
        string_list = list()
        value_list = list()
        for field_name in self.__class__.__dict__['Meta'].__dict__['fields']:
            value_list.append(getattr(self, field_name))
            string_list.append(
                '`{0}` {1} {2} {3}'.format(
                    field_name, 
                    types[type(self.__class__.__dict__[field_name])],
                    is_req[self.__class__.__dict__[field_name].is_required],
                    is_prim[self.__class__.__dict__[field_name].is_primary],
                )
            )
        string_list = ','.join(string_list)
        cursor.execute(
            'CREATE TABLE IF NOT EXISTS `{0}` ({1})'.format(
                getattr(self,'Meta').__dict__['table_name'],
                string_list
            )
        )
        
        value_list = [str(add_(change_boolean(e))) for e in value_list]
        value_list = ','.join(value_list)
        query_insert = """INSERT INTO {0} VALUES ({1})""".format(
            self.__class__.__dict__['Meta'].__dict__['table_name'],
            value_list
        )
        cursor.execute(query_insert)
        db.commit()
        db.close()
        gc.collect()

In [12]:
class User(Model):
    
    TelNumber = IntField('TelNumber',required=True, default=2)
    Name =      CharField('Name',required=True, default='fdf')
    Sex =       BoolField('Sex',required=True, default=True)
    Money  =    FloatField('Money',required=False, default=0.0)
    
    def __init__(self, **kwarg):
        super().__init__(**kwarg)
    
    def update_money(self, value):
        self.Money = value

class Stuff(User):
    Post = IntField(required=True, default = -1)
    
a = Stuff.objects.create(id=0,TelNumber=322188, Name='Alan', Sex=False, Money=630.0)
a.save()
a.TelNumber = 232916
a.save()

id 0
UPDATE Stuff SET Post=-1,TelNumber=322188,Name="Alan",Sex=0,Money=630.0 WHERE id=0
id 0
UPDATE Stuff SET Post=-1,TelNumber=232916,Name="Alan",Sex=0,Money=630.0 WHERE id=0


In [335]:
class A():
    a = 0
    b = ""
    c = True
    
    def __init__(self, a, b, c):
        print("Called fron {}".format(self))
        self.a = a
        self.b = b
        self.c = c
        
    def update_a(self, a):
        print("Called fron {}".format(self))
        self.a = a
    
    def update_b(self, b):
        print("Called fron {}".format(self))
        self.b = b
        
    def update_c(self, c):
        print("Called fron {}".format(self))
        self.c = c
        
class MetaClass(type):
    def __new__(mcs, name, bases, attr):
        meta = attr.get('Meta')
        print(attr)
        if meta is not None:
            print(meta)
            print(hasattr(attr['Meta'], 'table_name'))
        return super().__new__(mcs, name, bases, attr)
        
class B(A):
    
    class Meta:
        table_name = 'B'
        fields = ('d')
        
    a = 'a'
    
    def __init__(self, a, b, c, d):
        print("Called fron {}".format(self))
        self.a = d
        super().__init__(a, b, c)
        
    def lel(self):
        return 1
    
    @property
    def ex(self):
        return self.a
        
        

In [43]:
b = B(1,2,3,4)
b.ex

Called fron <__main__.B object at 0x7f8b1e5c5eb8>
Called fron <__main__.B object at 0x7f8b1e5c5eb8>


1

In [82]:
B(1,2,3,4).__class__.__dict__['Meta'].__dict__['table_name']

Called fron <__main__.B object at 0x7ff93c273588>
Called fron <__main__.B object at 0x7ff93c273588>


'B'

In [88]:
a= {'a': 1, 'b' : 2}
set(a.keys())

{'a', 'b'}

In [86]:
a = B(1,2,3,4)
isinstance(B.mro()[0], MetaClass)

Called fron <__main__.B object at 0x7ff93c2210b8>
Called fron <__main__.B object at 0x7ff93c2210b8>


True

In [136]:
-2.23e-308
        

-2.23e-308

In [15]:
import sys
print(sys.version)

2.7.15 |Anaconda, Inc.| (default, Dec 14 2018, 19:04:19) 
[GCC 7.3.0]


In [14]:
class A:
    pass
        
a = [A(), A(), A()]
dir(a.__class__)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__delitem__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__reversed__',
 '__rmul__',
 '__setattr__',
 '__setitem__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 'append',
 'clear',
 'copy',
 'count',
 'extend',
 'index',
 'insert',
 'pop',
 'remove',
 'reverse',
 'sort']

In [63]:
len({1: 'sd', 2: 'da'})

2

In [273]:
a = dict()
a

True

In [359]:
2**31

2147483648

In [20]:
s = 'adas'
'"' + s +'"'

'"adas"'

In [27]:
a= "{0}"
a

'{0}'

In [58]:
'SELECT * FROM WHERE {0}={1}'.format('id', 1)

'SELECT * FROM WHERE id=1'